In [1]:
from pathlib import Path
USE_COLAB: bool = True
dataset_base_path = Path("/content/drive/My Drive/ECE 792 - Advance Topics in Machine Learning/Datasets/RealFaces/CelebA")
if USE_COLAB:
  from google.colab import drive
  
  # Mount the drive to access google shared docs
  drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [2]:
from typing import Tuple, Union, List, Optional, Callable
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import OneHotEncoder
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
from tqdm import tqdm
from dataclasses import dataclass
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from torchvision.utils import save_image, make_grid
import torchvision.transforms as transforms
import torch.optim as optim
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
from tqdm import tqdm
from zipfile import ZipFile
from PIL import Image

In [3]:
# Ex.
# gen = Generator()
# gen.apply(weights_init)
# apply normal distribution with mean of 0 and standard deviation of 0.02 to model weights
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [4]:
class CelebrityData(torch.utils.data.Dataset):

  def __init__(self,zip_file,transform = None) :
    self.zip_file = zip_file
    self.transform = transform

    # Create a ZipFile Object and load sample.zip in it
    with ZipFile(str(zip_file), 'r') as zipObj:
      # Extract all the contents of zip file in current directory
      zipObj.extractall()


    self.ListOfImages = zipObj.namelist()[1:]


  def __getitem__(self,index):

    image = Image.open(self.ListOfImages[index]).convert('RGB')

    if self.transform is not None:
      image = self.transform(image)

    return image


  def __len__(self):
    return len(self.ListOfImages)


class CelebrityDataCVAE(CelebrityData):
  def __init__(self, zip_file: Path, attr_file: Path, img_transform=None, seed=None):
    super().__init__(zip_file, img_transform)
    self.attrs = pd.read_csv(str(attr_file), index_col=0)

  def __getitem__(self, index):
    img_path = self.ListOfImages[index]
    img = Image.open(img_path).convert("RGB")
    if self.transform is not None:
      img = self.transform(img)

    # attr = torch.Tensor(self.attrs.loc[int(Path(img_path).with_suffix("").name)])
    attr = self.get_attr(int(Path(img_path).with_suffix("").name))

    return img, attr

  def get_attr(self, index):
    return torch.Tensor(self.attrs.loc[index])

In [5]:
output_dir = Path("/content/drive/My Drive/ECE 792 - Advance Topics in Machine Learning/Code/DatasetGeneration/CDCGAN/models")
output_dir.mkdir(exist_ok=True, parents=True)
zip_file = dataset_base_path.parent / "img_align_celeb.zip"
attr_file = dataset_base_path / "img_align_celeba_attr.csv"

# learning rate & beta1 of Adam optimizer from original DCGAN paper
config = {
    "batch_size": 128,
    "epochs": 50,
    "latent_dim": 64,
    "lr": 2e-4,
    "betas": (0.5, 0.999),
    "chkp_freq": 1,  # frequency of epochs to save outputs
    "n_sampled_imgs": 25,
    "img_size": 64,
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
celebrity_data = CelebrityDataCVAE(zip_file = zip_file,
                                   attr_file = attr_file,
                                   img_transform = transforms.Compose([
                                      # Important parts, above can be ignored
                                      transforms.Resize(int(config['img_size'] * 1.1)),
                                      transforms.CenterCrop(config['img_size']),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean = (0.5,0.5,0.5),
                                                           std  = (0.5,0.5,0.5))
]))

dataloader = torch.utils.data.DataLoader(dataset = celebrity_data,
                                         shuffle = True,
                                         batch_size = config['batch_size'],
                                         num_workers = 2,
                                         drop_last = True,
                                         pin_memory = True) 

In [7]:
class Generator(nn.Module):
  def __init__(self):
    super(Generator, self).__init__()
    self.deconv = nn.Sequential(
        nn.ConvTranspose2d(in_channels=104, out_channels=256, kernel_size=(3, 3), stride=(2, 2), padding=(0, 0)),
        nn.BatchNorm2d(256),
        nn.Tanh(),

        nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=(3, 3), stride=(2, 2), padding=(0, 0)),
        nn.BatchNorm2d(128),
        nn.Tanh(),

        nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=(3, 3), stride=(2, 2), padding=(0, 0)),
        nn.BatchNorm2d(64),
        nn.Tanh(),

        nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=(4, 4), stride=(2, 2), padding=(0, 0)),
        nn.BatchNorm2d(32),
        nn.Tanh(),

        nn.ConvTranspose2d(in_channels=32, out_channels=3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1)),
        # nn.BatchNorm2d(3),
        nn.ReLU(),
    )

  def forward(self, x):
    return self.deconv(x)


class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator, self).__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(in_channels=43, out_channels=96, kernel_size=(4, 4), stride=(2, 2), padding=(0, 0)),
        nn.BatchNorm2d(96),
        nn.LeakyReLU(0.2),

        nn.Conv2d(in_channels=96, out_channels=128, kernel_size=(4, 4), stride=(2, 2), padding=(0, 0)),
        nn.BatchNorm2d(128),
        nn.LeakyReLU(0.2),

        nn.Conv2d(in_channels=128, out_channels=64, kernel_size=(4, 4), stride=(2, 2), padding=(0, 0)),
        nn.BatchNorm2d(64),
        nn.LeakyReLU(0.2),

        nn.Conv2d(in_channels=64, out_channels=1, kernel_size=(4, 4), stride=(3, 3), padding=(0, 0)),
        # nn.BatchNorm2d(1),
        nn.Sigmoid(),
    )

  def forward(self, x):
    return self.conv(x)

In [8]:
# use binary cross-entropy loss
criterion = nn.BCELoss()

# labels for real images are '1' and labels for fake images are '0'
def label_real(batch_size: int) -> torch.Tensor:
  return torch.ones(batch_size,)
def label_fake(batch_size: int) -> torch.Tensor:
  return torch.zeros(batch_size,)

# generate our random noise to sample from for our generator
def generate_z_noise(batch_size: int, latent_dim: int) -> torch.Tensor:
  return torch.randn(batch_size, latent_dim)

In [9]:
# function to plot discriminator loss vs generator loss
def gan_loss_discriminator_vs_generator_plot(disc_loss, gen_loss, output_path):
  plot_labels = ["discriminator", "generator"]
  x_epochs = np.arange(1, len(disc_loss) + 1)
  plt.plot(x_epochs, disc_loss, label=plot_labels[0])
  plt.plot(x_epochs, gen_loss, label=plot_labels[1])
  plt.title("Discriminator vs Generator Loss")
  plt.xlabel("Epochs")
  plt.ylabel("Loss")
  plt.legend()
  plt.savefig(str(output_path))
  plt.close()

In [10]:
# reshape attributes into square matrices
def reshape_labels_to_repeat_attrs_into_square_matrix(
  labels: torch.Tensor,
  square_shape: Tuple[int, int] = (config["img_size"], config["img_size"]),
) -> torch.Tensor:
  return torch.repeat_interleave(
      torch.repeat_interleave(
          labels, square_shape[0]
      ), square_shape[1]
  ).reshape(labels.shape[0], labels.shape[1], square_shape[0], square_shape[1])

In [11]:
# initialize generator & discriminator
generator = Generator().to(device)
discriminator = Discriminator().to(device)

# create optimizers for generator & discriminator
optim_g = torch.optim.Adam(generator.parameters(), lr=config["lr"], betas=config["betas"])
optim_d = torch.optim.Adam(discriminator.parameters(), lr=config["lr"], betas=config["betas"])

# use fixed noise for outputting samples during each epoch
fixed_noise = generate_z_noise(config["n_sampled_imgs"], config["latent_dim"]).to(device)
attr_indices = torch.randint(low=1, high=len(celebrity_data), size=(config["n_sampled_imgs"], 1))
fixed_attrs = torch.Tensor(config["n_sampled_imgs"], 40)
for idx, attr_idx in enumerate(attr_indices):
  fixed_attrs[idx] = celebrity_data.get_attr(attr_idx.item()).to(device)
noise_gen_input = torch.concat([fixed_noise, fixed_attrs.to(device)], dim=1).unsqueeze(-1).unsqueeze(-1).to(device)

# ensure models are in training mode
# initialize weights of models with normal distribution
generator.train()
generator.apply(weights_init)
discriminator.train()
discriminator.apply(weights_init)

# output paths to retain pertinent information
base_imgs_output_path = output_dir / "images"
base_imgs_output_path.mkdir(exist_ok=True, parents=True)
base_loss_output_path = output_dir / "loss"
base_loss_output_path.mkdir(exist_ok=True, parents=True)
base_models_output_path = output_dir / "models"
base_models_output_path.mkdir(exist_ok=True, parents=True)

# keep track of generator & discriminator losses
losses_generator = []
losses_discriminator = []
epoch_tqdm = tqdm(total=config["epochs"], position=1)
print("Starting Training...")
for epoch in range(config["epochs"]):
  loss_g = 0.
  loss_d_real = 0.
  loss_d_fake = 0.
  epoch_img_output_path = base_imgs_output_path / str(epoch+1)
  epoch_img_output_path.mkdir(exist_ok=True, parents=True)
  model_output_path = base_models_output_path / f"CDCGAN--{epoch+1}.pth"
  loss_output_path = base_loss_output_path / f"CDCGAN-loss--{epoch+1}.png"

  for batch_idx, (img, attr) in enumerate(dataloader):
    img = img.to(device)
    attr = attr.to(device)
    # reshape attributes for concatenation for discriminator input
    attr_reshaped = reshape_labels_to_repeat_attrs_into_square_matrix(attr)

    # generate noise input with concatenated attributes
    z = generate_z_noise(config["batch_size"], config["latent_dim"]).to(device)
    gen_input = torch.concat([z, attr], dim=1).unsqueeze(-1).unsqueeze(-1).to(device)

    # Step 1, train discriminator
    # produce fake image outputs from generator
    gen_fake = generator(gen_input).detach()

    # concatenate images with reshaped labels for input to discriminator
    gen_fake_cat = torch.concat([gen_fake, attr_reshaped], dim=1)
    img_real_cat = torch.concat([img, attr_reshaped], dim=1)

    # target decisions to be used in binary cross entropy loss
    label_real_ = label_real(config["batch_size"]).to(device)
    label_fake_ = label_fake(config["batch_size"]).to(device)

    # zero out weights in optimizer
    optim_d.zero_grad()

    # get outputs from discriminator using real images & calculate loss
    disc_out_real = discriminator(img_real_cat).view(-1)
    loss_real = criterion(disc_out_real, label_real_)
    
    # get outputs from discriminator using fake images & calculate loss
    disc_out_fake = discriminator(gen_fake_cat).view(-1)
    loss_fake = criterion(disc_out_fake, label_fake_)

    loss_D = 0.5 * (loss_fake + loss_real)
    loss_D.backward()
    optim_d.step()

    loss_d_real += loss_real
    loss_d_fake += loss_fake

    # Step 2, train generator
    optim_g.zero_grad()

    gen_fake = generator(gen_input)
    gen_fake_cat = torch.concat([gen_fake, attr_reshaped], dim=1)

    disc_out_fake_g_z = discriminator(gen_fake_cat).view(-1)

    loss_gen = criterion(disc_out_fake_g_z, label_real_)
    loss_gen.backward()

    optim_g.step()

    loss_g += loss_gen

    if (batch_idx + 1) % 50 == 0:
      print("Epoch: [%d/%d] [%d/%d]\tLoss_G: %.6f\tLoss_D_fake: %.6f\tLoss_D_real: %.6f" % (
          epoch+1, config["epochs"], batch_idx + 1, len(dataloader), 
          float(loss_g / batch_idx), float(loss_d_fake / batch_idx), float(loss_d_real / batch_idx)
      ))
  
  epoch_tqdm.update(1)
  losses_generator.append(float(loss_g / len(dataloader)))
  losses_discriminator.append(
      float(
          (loss_d_real / len(dataloader)) + (loss_d_fake / len(dataloader))
      ) / 2
  )
  print("Loss_G: %.6f\tLoss_D: %.6f" % (
      losses_generator[-1], losses_discriminator[-1]
  ))

  if (epoch + 1) % config["chkp_freq"] == 0:
    torch.save(
        {
            "Generator": generator.state_dict(),
            "Discriminator": discriminator.state_dict(),
        },
        str(model_output_path)
    )
    gan_loss_discriminator_vs_generator_plot(losses_discriminator, losses_generator, loss_output_path)

    generated_imgs = generator(noise_gen_input)
    for idx, gen_img in enumerate(generated_imgs):
      img_output_path = epoch_img_output_path / f"{idx}.png"
      save_image(gen_img, img_output_path)


  0%|          | 0/50 [00:00<?, ?it/s]

Starting Training...
Epoch: [1/50] [50/1582]	Loss_G: 1.172011	Loss_D_fake: 0.559207	Loss_D_real: 0.484641
Epoch: [1/50] [100/1582]	Loss_G: 1.645914	Loss_D_fake: 0.419441	Loss_D_real: 0.419761
Epoch: [1/50] [150/1582]	Loss_G: 1.849018	Loss_D_fake: 0.389767	Loss_D_real: 0.395787
Epoch: [1/50] [200/1582]	Loss_G: 2.006782	Loss_D_fake: 0.343406	Loss_D_real: 0.370839
Epoch: [1/50] [250/1582]	Loss_G: 2.080698	Loss_D_fake: 0.337219	Loss_D_real: 0.352319
Epoch: [1/50] [300/1582]	Loss_G: 2.108935	Loss_D_fake: 0.333281	Loss_D_real: 0.344960
Epoch: [1/50] [350/1582]	Loss_G: 2.085804	Loss_D_fake: 0.348072	Loss_D_real: 0.365617
Epoch: [1/50] [400/1582]	Loss_G: 2.078799	Loss_D_fake: 0.352873	Loss_D_real: 0.373503
Epoch: [1/50] [450/1582]	Loss_G: 2.065926	Loss_D_fake: 0.356361	Loss_D_real: 0.381801
Epoch: [1/50] [500/1582]	Loss_G: 2.044218	Loss_D_fake: 0.358845	Loss_D_real: 0.384993
Epoch: [1/50] [550/1582]	Loss_G: 2.025642	Loss_D_fake: 0.363471	Loss_D_real: 0.390166
Epoch: [1/50] [600/1582]	Loss_G: 1


  2%|▏         | 1/50 [03:02<2:28:49, 182.23s/it]

Loss_G: 1.486315	Loss_D: 0.538844
Epoch: [2/50] [50/1582]	Loss_G: 1.024755	Loss_D_fake: 0.656195	Loss_D_real: 0.703431
Epoch: [2/50] [100/1582]	Loss_G: 1.009911	Loss_D_fake: 0.657248	Loss_D_real: 0.669973
Epoch: [2/50] [150/1582]	Loss_G: 1.008561	Loss_D_fake: 0.649450	Loss_D_real: 0.669629
Epoch: [2/50] [200/1582]	Loss_G: 1.005772	Loss_D_fake: 0.650625	Loss_D_real: 0.668328
Epoch: [2/50] [250/1582]	Loss_G: 1.001468	Loss_D_fake: 0.652879	Loss_D_real: 0.668849
Epoch: [2/50] [300/1582]	Loss_G: 1.003078	Loss_D_fake: 0.650657	Loss_D_real: 0.667813
Epoch: [2/50] [350/1582]	Loss_G: 0.996268	Loss_D_fake: 0.651522	Loss_D_real: 0.668655
Epoch: [2/50] [400/1582]	Loss_G: 0.992695	Loss_D_fake: 0.651663	Loss_D_real: 0.667201
Epoch: [2/50] [450/1582]	Loss_G: 0.993553	Loss_D_fake: 0.652379	Loss_D_real: 0.666546
Epoch: [2/50] [500/1582]	Loss_G: 0.993105	Loss_D_fake: 0.651716	Loss_D_real: 0.668377
Epoch: [2/50] [550/1582]	Loss_G: 0.989611	Loss_D_fake: 0.653206	Loss_D_real: 0.668333
Epoch: [2/50] [600/15


  4%|▍         | 2/50 [05:59<2:23:30, 179.38s/it]

...


 84%|████████▍ | 42/50 [2:03:52<23:28, 176.00s/it]

Loss_G: 6.212861	Loss_D: 0.027450
Epoch: [43/50] [50/1582]	Loss_G: 7.004888	Loss_D_fake: 0.019021	Loss_D_real: 0.035060
Epoch: [43/50] [100/1582]	Loss_G: 6.625739	Loss_D_fake: 0.016627	Loss_D_real: 0.031382
Epoch: [43/50] [150/1582]	Loss_G: 7.166782	Loss_D_fake: 0.038249	Loss_D_real: 0.069506
Epoch: [43/50] [200/1582]	Loss_G: 7.075598	Loss_D_fake: 0.038139	Loss_D_real: 0.068617
Epoch: [43/50] [250/1582]	Loss_G: 6.935509	Loss_D_fake: 0.035788	Loss_D_real: 0.061806
Epoch: [43/50] [300/1582]	Loss_G: 6.737893	Loss_D_fake: 0.033600	Loss_D_real: 0.055548
Epoch: [43/50] [350/1582]	Loss_G: 6.572298	Loss_D_fake: 0.031571	Loss_D_real: 0.051594
Epoch: [43/50] [400/1582]	Loss_G: 6.494100	Loss_D_fake: 0.030182	Loss_D_real: 0.049600
Epoch: [43/50] [450/1582]	Loss_G: 6.494502	Loss_D_fake: 0.029670	Loss_D_real: 0.049285
Epoch: [43/50] [500/1582]	Loss_G: 6.453635	Loss_D_fake: 0.028193	Loss_D_real: 0.047693
Epoch: [43/50] [550/1582]	Loss_G: 6.437588	Loss_D_fake: 0.027635	Loss_D_real: 0.046762
Epoch: [43


 86%|████████▌ | 43/50 [2:06:49<20:33, 176.22s/it]

Loss_G: 6.557028	Loss_D: 0.027120
Epoch: [44/50] [50/1582]	Loss_G: 6.684844	Loss_D_fake: 0.026163	Loss_D_real: 0.044352
Epoch: [44/50] [100/1582]	Loss_G: 6.500250	Loss_D_fake: 0.023891	Loss_D_real: 0.035535
Epoch: [44/50] [150/1582]	Loss_G: 6.441092	Loss_D_fake: 0.023345	Loss_D_real: 0.033878
Epoch: [44/50] [200/1582]	Loss_G: 6.292917	Loss_D_fake: 0.023176	Loss_D_real: 0.032560
Epoch: [44/50] [250/1582]	Loss_G: 6.310370	Loss_D_fake: 0.022701	Loss_D_real: 0.033076
Epoch: [44/50] [300/1582]	Loss_G: 6.569055	Loss_D_fake: 0.033116	Loss_D_real: 0.060651
Epoch: [44/50] [350/1582]	Loss_G: 6.612257	Loss_D_fake: 0.041591	Loss_D_real: 0.077817


KeyboardInterrupt: ignored